## 1. Age

According to Santander Cycles Survey of Q2 2017/2018 their main target group depicts of people of between the age 16 and 54.<br>
They group their customers in the following age groups:

![](img/target.png)

As this plot shows they have more male users than female users, why we give maless a higher weight than females.<br>
Moreover we should weight the several age groups differently. 

Population data of 2017 was found at the [Office for National Statistics](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland).

After preparing the data in a first way we have the age groups ordered by London districts:
### To Do:
* find data for 2018,2019
* assign districts to coordinates
* assign age to stations

In [1]:
import pandas as pd
import numpy as np

In [2]:
age = pd.read_csv("age.csv")
age.head()

,Name,All ages,25-34,35-44,45-54,55-64,>65
0,Camden,253361,54527,39552,30599,21676,30098
1,City of London,7654,1050,1002,1255,1024,1482
2,Hackney,275929,70473,49206,31106,20243,20394
3,Hammersmith and Fulham,182998,40080,30736,24184,15855,19334
4,Haringey,271224,52469,48048,36933,24046,26628


## 2. Events
Extracted events from 2015-2019 from [Lodon Events](http://www.londontown.com/London/London-Events-2019).</br>
### To Do:
* extract days of datetime
* find coordinates to postalcodes
* assign events to stations

In [3]:
import json
import pandas as pd
with open('eventsdata/events-2015.json', 'r') as f:
    data15 = json.load(f)
dfRaw15= pd.DataFrame(data15)
df15=dfRaw15.assign(Year='2015')
df1 =df15.drop(['locality','url'],axis=1)
df1.head()

,dates,postcode,street,title,Year
0,05th - 06th June 2018,KT8 9AU,"East Molesey, Surrey",\nLionel Richie,2015
1,08th - 09th March 2019,SE1 8XX,Belvedere Road,\nWomen of the World Festival,2015
2,02nd - 14th March 2015,SW1E 5JA,12 Palace Street,\nRuby Wax: Sane New World,2015
3,03rd - 07th March 2015,TW9 1QJ,"The Green, Surrey",\nHarvey,2015
4,03rd - 14th March 2015,NW1 3BF,"15-16 Triton Street, Regents Place",\nCitizen Puppet,2015


In [4]:
with open('eventsdata/events-2016.json', 'r') as f:
    data16 = json.load(f)
dfRaw16= pd.DataFrame(data16)
df16=dfRaw16.assign(Year='2016')
df2 = df16.drop(['locality','url'],axis=1)
df2.head()

,dates,postcode,street,title,Year
0,01st July 2016,W2 2UH,Rangers Lodge,\nMassive Attack,2016
1,01st July - 06th August 2016,SW1W 8AS,Sloane Square,\nUnreachable,2016
2,"12th, 13th and 14th July 2019",W2 2UH,Rangers Lodge,\nBarclaycard presents British Summer Time Hyd...,2016
3,02nd July - 02nd October 2016,W1J 0BD,Burlington House,\nDavid Hockney RA: 79 Portraits and 2 Still L...,2016
4,02nd July 2016,W2 2UH,Rangers Lodge,"\nFlorence & The Machine, Kendrick Lamar, Jami...",2016


In [5]:
with open('eventsdata/events-2017.json', 'r') as f:
    data17 = json.load(f)
dfRaw17= pd.DataFrame(data17)
df17=dfRaw17.assign(Year='2017')
df3 = df17.drop(['locality','url'],axis=1)
df3.head()

,dates,postcode,street,title,Year
0,01st March - 03rd June 2017,WC2H 0HH,2 Charing Cross Road,\nThe Miser,2017
1,01st - 10th March 2017,SW7 2AP,Kensington Gore,\nRussell Howard: Round The World,2017
2,01st March - 17th June 2017,WC2R 0NH,404 Strand,\nStepping Out,2017
3,01st March - 15th October 2017,SE1 9PP,Upper Ground,\nThe Art of the Brick: DC Super Heroes,2017
4,02nd March - 15th April 2017,WC2H 9LD,"41 Earlham Street, Seven Dials",\nLimehouse,2017


In [6]:
with open('eventsdata/events-2018.json', 'r') as f:
    data18 = json.load(f)
dfRaw18= pd.DataFrame(data18)
df18=dfRaw18.assign(Year='2018')
df4 = df18.drop(['locality','url'],axis=1)
df4.head()

,dates,postcode,street,title,Year
0,01st March - 06th May 2018,EC1V 3SB,London,\nSOMNAI,2018
1,01st March 2018,SW9 9SL,211 Stockwell Road,\nMorrissey,2018
2,01st March - 20th May 2018,WC2H 0HE,St Martin's Place,\nVictorian Giants: The Birth Of Art Photography,2018
3,02nd March - 09th September 2018,SW1Y 4RL,63-65 Haymarket,\nBrief Encounter,2018
4,02nd March - 05th May 2018,SE1 8LZ,66 The Cut,\nThe Inheritance,2018


In [7]:
with open('eventsdata/events-2019.json', 'r') as f:
    data19 = json.load(f)
dfRaw19= pd.DataFrame(data19)
df19=dfRaw19.assign(Year='2019')
df5 = df19.drop(['locality','url'],axis=1)
df5.head()

,dates,postcode,street,title,Year
0,01st - 02nd March 2019,EC1R 4TN,Rosebery Avenue,\nRichard Alston Dance Company: Quartermark,2019
1,01st - 31st March 2019,W4 2RP,Burlington Lane,\nChiswick House Camellia Festival,2019
2,3rd March to 2nd June 2019,W1J 0BD,Burlington House,\nThe Renaissance Nude,2019
3,04th - 09th March 2019,TW9 1QJ,"The Green, Surrey",\nArt,2019
4,05th - 09th March 2019,EC1R 4TN,Rosebery Avenue,\nBalletBoyz: Them/Us,2019


In [33]:
frames = [df1, df2, df3,df4,df5]

final = pd.concat(frames)
f = final.assign(Month='0')
f

,dates,postcode,street,title,Year,Month
0,05th - 06th June 2018,KT8 9AU,"East Molesey, Surrey",\nLionel Richie,2015,0
1,08th - 09th March 2019,SE1 8XX,Belvedere Road,\nWomen of the World Festival,2015,0
2,02nd - 14th March 2015,SW1E 5JA,12 Palace Street,\nRuby Wax: Sane New World,2015,0
3,03rd - 07th March 2015,TW9 1QJ,"The Green, Surrey",\nHarvey,2015,0
4,03rd - 14th March 2015,NW1 3BF,"15-16 Triton Street, Regents Place",\nCitizen Puppet,2015,0
5,04th - 28th March 2015,EC2Y 8DS,Silk Street,\nAntigone,2015,0
6,06th - 15th September 2018,SW11 5TN,Lavender Hill,\nMissing,2015,0
7,04th March - 31st May 2015,WC2N 5DN,Trafalgar Square,\nInventing Impressionism,2015,0
8,04th March - 06th September 2015,SW3 4SQ,"Duke of York's Square, King's Road",\nPangaea II: New Art from Africa and Latin Am...,2015,0
9,05th March - 31st August 2015,SE1 6HZ,"Lambeth Road, Elephant and Castle",\nFashion on the Ration,2015,0


In [54]:
f.loc[f['dates'].str.contains('January'), 'Month'] = 1
f.loc[f['dates'].str.contains('February'), 'Month'] = 2
f.loc[f['dates'].str.contains('March'), 'Month'] = 3
f.loc[f['dates'].str.contains('April'), 'Month'] = 4
f.loc[f['dates'].str.contains('May'), 'Month'] = 5
f.loc[f['dates'].str.contains('June'), 'Month'] = 6
f.loc[f['dates'].str.contains('July'), 'Month'] = 7
f.loc[f['dates'].str.contains('August'), 'Month'] = 8
f.loc[f['dates'].str.contains('September'), 'Month'] = 9
f.loc[f['dates'].str.contains('October'), 'Month'] = 10
f.loc[f['dates'].str.contains('November'), 'Month'] = 11
f.loc[f['dates'].str.contains('December'), 'Month'] = 12

f.loc[f['dates'].str.contains('2015'), 'Year'] = 2015
f.loc[f['dates'].str.contains('2016'), 'Year'] = 2016
f.loc[f['dates'].str.contains('2017'), 'Year'] = 2017
f.loc[f['dates'].str.contains('2018'), 'Year'] = 2018
f.loc[f['dates'].str.contains('2019'), 'Year'] = 2019

In [10]:
f.shape

(1817, 6)

In [58]:
date = f['dates']
str(date)
days= []
dash = '-'
d = '0'

for index, row in date.iteritems():
    #if np.where(date.applymap(lambda x: x == '-')):
    #if date.apply(lambda x: any(pd.Series(x).str.contains('-'))):
    # if date.str.contains('-'):
  
    if dash in row and len(row)<28:
        b= row[7:9]
        a = row[0:2]
        c=a+b
        days.append(c)
        
    elif dash in row and len(row)>28:
        x= row[:16]
        days.append(x)
    else:
        b = row[:2]
        days.append(b)
        
        #if _type in result:
                    #values[_type].append(result[_type])
f=f.assign(Days = days)
print(days)
#df = pd.DataFrame({'Days': days})
len(days)
#df.to_csv('days.csv')

[u'0506', u'0809', u'0214', u'0307', u'0314', u'0428', u'0615', u'04rc', u'04th March - 06t', u'05th March - 31s', u'0621', u'0529', u'0506', u'05th March - 31s', u'2425', u'06', u'06', u'0708', u'0708', u'17', u'08', u'08', u'09th March - 05t', u'0915', u'1015', u'10', u'10rc', u'11', u'11rc', u'1821', u'12rc', u'12rc', u'1220', u'12', u'13rc', u'13', u'1315', u'13', u'14rc', u'14th March - 02n', u'18', u'28', u'16', u'1718', u'2831', u'1822', u'0915', u'0716', u'1821', u'08th February - ', u'21', u'20', u'20th March - 11t', u'2022', u'2021', u'17', u'21', u'2129', u'19', u'23', u'24rc', u'24rc', u'2426', u'2528', u'25', u'2527', u'25rc', u'2629', u'27th March - 13t', u'27', u'27', u'28', u'28th March - 31s', u'28th March - 28t', u'28rc', u'30', u'01st April - 31s', u'01ri', u'01ri', u'01', u'01st April - 13t', u'01st April - 31s', u'02nd April - 25t', u'29', u'02', u'2427', u'29th March - 16t', u'03', u'0405', u'0411', u'04', u'06', u'07ri', u'0809', u'09th April - 30t', u'07ri', u'0

1817

In [59]:
f

,dates,postcode,street,title,Year,Month,Days
0,05th - 06th June 2018,KT8 9AU,"East Molesey, Surrey",\nLionel Richie,2018,6,0506
1,08th - 09th March 2019,SE1 8XX,Belvedere Road,\nWomen of the World Festival,2019,3,0809
2,02nd - 14th March 2015,SW1E 5JA,12 Palace Street,\nRuby Wax: Sane New World,2015,3,0214
3,03rd - 07th March 2015,TW9 1QJ,"The Green, Surrey",\nHarvey,2015,3,0307
4,03rd - 14th March 2015,NW1 3BF,"15-16 Triton Street, Regents Place",\nCitizen Puppet,2015,3,0314
5,04th - 28th March 2015,EC2Y 8DS,Silk Street,\nAntigone,2015,3,0428
6,06th - 15th September 2018,SW11 5TN,Lavender Hill,\nMissing,2018,9,0615
7,04th March - 31st May 2015,WC2N 5DN,Trafalgar Square,\nInventing Impressionism,2015,5,04rc
8,04th March - 06th September 2015,SW3 4SQ,"Duke of York's Square, King's Road",\nPangaea II: New Art from Africa and Latin Am...,2015,9,04th March - 06t
9,05th March - 31st August 2015,SE1 6HZ,"Lambeth Road, Elephant and Castle",\nFashion on the Ration,2015,8,05th March - 31s


In [60]:
#f.to_csv('newdata.csv')

In [ ]:
stations = pd.read_csv('/home/kronork/mapReduce/raw/rental_stations_saved.csv')

In [ ]:
stations.head()

## 3. Earnings
Extracted earnings in Weekly Gross Pay (£) from 2015-2019 from [Office for National Statistics](https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/annualsurveyofhoursandearnings/2016provisionalresults).</br>
### To Do:
* assign districts to coordinates
* assign earnings to stations

In [ ]:
earnings = pd.read_csv('/home/kronork/mapReduce/Data_Profiling/earnings.csv')
earnings.head()